## LLM from scratch

- 원본소스 : [홍정모 연구소 youtube](https://youtu.be/osv2csoHVAo?si=hE74wbahXyvVgCTc)


### 데이터 전처리

여기서는 줄바꿈을 빈칸으로, 여러빈칸을 하나의 빈칸으로 변경

In [5]:
import re
from pathlib import Path

data_dir = Path('./data/HP')
data_dir.mkdir(parents=True, exist_ok=True)


def clean_text(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        book_text = file.read()

    cleaned_text = re.sub(r'\n+', ' ', book_text) # 줄바꿈을 빈칸으로 변경
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text) # 여러 빈칸을 하나의 빈칸으로

    base_filename = filename.name
    print("cleaned_" + base_filename, len(cleaned_text), "characters") # 글자 수 출력

    output_path = data_dir / f"cleaned_{base_filename}"

    with open(output_path, 'w', encoding='utf-8') as file:
        file.write(cleaned_text)

# 파일을 찾을 폴더 경로를 Path 객체로 생성
print(f"'{data_dir}' 폴더에서 .txt 파일을 검색합니다...")

# Path 객체의 glob 메서드를 사용하여 .txt 파일을 찾습니다.
# 결과는 제너레이터(generator)이므로 list()로 변환해줍니다.
try:
    filenames_list = list(data_dir.glob('*.txt'))

    # 결과 출력
    if filenames_list:
        print("\n[성공] 발견된 .txt 파일 목록:")
        for file_path in filenames_list:
            # Path 객체는 그대로 출력해도 경로가 잘 보입니다.
            # 문자열이 필요하면 str(file_path)로 변환할 수 있습니다.
            print(file_path)
    else:
        print(f"\n[알림] 해당 경로에 .txt 파일이 없습니다: {data_dir}")

except FileNotFoundError:
    print(f"\n[오류] 폴더를 찾을 수 없습니다: {data_dir}")

# filenames_list = ["02 Harry Potter and the Chamber of Secrets.txt"]

for filename in filenames_list:
    clean_text(filename)

'data/HP' 폴더에서 .txt 파일을 검색합니다...

[성공] 발견된 .txt 파일 목록:
data/HP/07 Harry Potter and the Deathly Hallows.txt
data/HP/05 Harry Potter and the Order of the Phoenix.txt
data/HP/02 Harry Potter and the Chamber of Secrets.txt
data/HP/06 Harry Potter and the Half-Blood Prince.txt
data/HP/03 Harry Potter and the Prisoner of Azkaban.txt
data/HP/01 Harry Potter and the Sorcerers Stone.txt
data/HP/04 Harry Potter and the Goblet of Fire.txt
cleaned_07 Harry Potter and the Deathly Hallows.txt 1133063 characters
cleaned_05 Harry Potter and the Order of the Phoenix.txt 1489734 characters
cleaned_02 Harry Potter and the Chamber of Secrets.txt 488771 characters
cleaned_06 Harry Potter and the Half-Blood Prince.txt 982041 characters
cleaned_03 Harry Potter and the Prisoner of Azkaban.txt 621137 characters
cleaned_01 Harry Potter and the Sorcerers Stone.txt 436000 characters
cleaned_04 Harry Potter and the Goblet of Fire.txt 1093670 characters


### 토큰화
UTF-8 Byte Pair Encoding(BPE)를 이용

In [7]:
# 토큰화 예제
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

text = "Hi! How are you? I am fine. Thank you."

tokens = tokenizer.encode(text)

print("글자수:", len(text), "토큰수", len(tokens))
print(tokens)
print(tokenizer.decode(tokens))
for t in tokens:
    print(f"{t}\t -> {tokenizer.decode([t])}")

글자수: 38 토큰수 13
[17250, 0, 1374, 389, 345, 30, 314, 716, 3734, 13, 6952, 345, 13]
Hi! How are you? I am fine. Thank you.
17250	 -> Hi
0	 -> !
1374	 ->  How
389	 ->  are
345	 ->  you
30	 -> ?
314	 ->  I
716	 ->  am
3734	 ->  fine
13	 -> .
6952	 ->  Thank
345	 ->  you
13	 -> .


In [10]:
# 한글 예제
from transformers import AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")
tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct")

text = "안녕하세요! 잘 지내고 계신가요? 저는 잘 지내고 있습니다. 감사합니다."

tokens = tokenizer.encode(text)

print("글자수:", len(text), "토큰수", len(tokens))
print(tokens)
print(tokenizer.decode(tokens))
for t in tokens:
    print(f"{t}\t -> {tokenizer.decode([t])}")


글자수: 40 토큰수 20
[3801, 1130, 5847, 362, 1353, 9158, 853, 25759, 392, 1229, 657, 1353, 9158, 853, 773, 50747, 375, 3365, 14010, 375]
안녕하세요! 잘 지내고 계신가요? 저는 잘 지내고 있습니다. 감사합니다.
3801	 -> 안녕
1130	 -> 하
5847	 -> 세요
362	 -> !
1353	 ->  잘
9158	 ->  지내
853	 -> 고
25759	 ->  계신가요
392	 -> ?
1229	 ->  저
657	 -> 는
1353	 ->  잘
9158	 ->  지내
853	 -> 고
773	 ->  있
50747	 -> 습니다
375	 -> .
3365	 ->  감사
14010	 -> 합니다
375	 -> .


### 데이터 로더

In [ ]:
# 데이터 로더 예제
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, txt, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # token_ids = tokenizer.encode("<|endoftext|>" + txt, allowed_special={"<|endoftext|>"})
        token_ids = tokenizer.encode(txt)

        print("# of tokens in txt:", len(token_ids))

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
    
# with open("cleaned_한글문서.txt", 'r', encoding='utf-8-sig') as file: # 선택: -sig를 붙여서 BOM 제거
with open("./data/HP/cleaned_02 Harry Potter and the Chamber of Secrets.txt", 'r', encoding='utf-8-sig') as file: # 선택: -sig를 붙여서 BOM 제거
    txt = file.read()

dataset = MyDataset(txt, max_length = 32, stride = 4)

train_loader = DataLoader(dataset, batch_size=128, shuffle=True, drop_last=True)


# of tokens in txt: 123176


In [13]:
dataiter = iter(train_loader)

x, y = next(dataiter)

print(tokenizer.decode(x[0].tolist()))
print(tokenizer.decode(y[0].tolist()))

’ll chuck you out of the house — and you haven’t got anywhere else to go — you haven’t got any friends to take you —”
ll chuck you out of the house — and you haven’t got anywhere else to go — you haven’t got any friends to take you —” “
